In [1]:
pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 16.2 MB/s eta 0:00:00


In [15]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

def load_data():
    # Datos del problema reducidos para ejemplo
    n = 10  # Número de clientes (incluyendo el depósito)
    vehicle_capacity = 200
    num_vehicles = 3

    # Datos de los clientes (coordenadas, demanda, tiempo de servicio, ventanas de tiempo)
    customers = [
        (40, 50, 0, 0, 1236, 0),    # Depósito
        (45, 68, 10, 912, 967, 90),
        (45, 70, 30, 825, 870, 90),
        (42, 66, 10, 65, 146, 90),
        (42, 68, 10, 727, 782, 90),
        (42, 65, 10, 15, 67, 90),
        (40, 69, 20, 621, 702, 90),
        (40, 66, 20, 170, 225, 90),
        (38, 68, 20, 255, 324, 90),
        (38, 70, 10, 534, 605, 90),
    ]

    return n, vehicle_capacity, num_vehicles, customers

def create_model(n, vehicle_capacity, num_vehicles, customers):
    x = [cust[0] for cust in customers]
    y = [cust[1] for cust in customers]
    demand = [cust[2] for cust in customers]
    ready_time = [cust[3] for cust in customers]
    due_date = [cust[4] for cust in customers]
    service_time = [cust[5] for cust in customers]

    # Crear matriz de distancias (en este caso se usan distancias Euclidianas para simplificar)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist[i][j] = np.sqrt((x[i] - x[j])**2 + (y[i] - y[j])**2)

    m = gp.Model('VRPTW')

    # Variables de decisión
    vars = {}
    for i in range(n):
        for j in range(n):
            if i != j:
                vars[i, j] = m.addVar(vtype=GRB.BINARY, obj=dist[i][j], name='e_%d_%d' % (i, j))

    # Tiempo de llegada a cada cliente
    arrivaltime = m.addVars(range(n), lb=ready_time, ub=due_date, vtype=GRB.CONTINUOUS, name='arrivaltime')

    # Restricciones
    m.addConstrs((vars[i, j] + vars[j, i] == 1 for i in range(n) for j in range(n) if i != j), name='no_subtour')

    # Restricciones de capacidad de los vehículos
    m.addConstrs((gp.quicksum(demand[i] * vars[i, j] for j in range(n) if i != j) <= vehicle_capacity for i in range(1, n)), name='capacity')

    # Restricciones de asignación y ventanas de tiempo
    for i in range(1, n):
        m.addConstr(gp.quicksum(vars[i, j] for j in range(n) if j != i) == 1)
        m.addConstr(gp.quicksum(vars[j, i] for j in range(n) if j != i) == 1)
        for j in range(1, n):
            if i != j:
                m.addConstr(arrivaltime[i] + service_time[i] + dist[i][j] <= arrivaltime[j] + (1 - vars[i, j]) * (due_date[i] + due_date[j]))

    # Configurar función objetivo y resolver
    m.ModelSense = GRB.MINIMIZE
    m.optimize()

    return m, vars

def print_solution(vars, n):
    solution = []
    for i in range(n):
        for j in range(n):
            if i != j and vars[i, j].x > 0.5:
                solution.append((i, j))

    print('Rutas:')
    for i, j in solution:
        print('Cliente %d -> Cliente %d' % (i, j))

def main():
    n, vehicle_capacity, num_vehicles, customers = load_data()
    m, vars = create_model(n, vehicle_capacity, num_vehicles, customers)
    print_solution(vars, n)

if __name__ == "__main__":
    main()


Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 189 rows, 100 columns and 639 nonzeros
Model fingerprint: 0x19bf98dc
Variable types: 10 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 2e+03]
Presolve removed 86 rows and 48 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: Unable to retrieve attribute 'x'